In [1]:
println("Start!!!")
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1500")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId
var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n,false)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n,false)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

Start!!!


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions prior to 1.8.
SLF4J: Ignoring binding found at [jar:file:/conda/envs/notebook/share/jupyter/kernels/scala211/pack/lib/slf4j-nop-1.7.26.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#ignoredBindings for an explanation.


Adding Hadoop conf dir /opt/meituan/hadoop/etc/hadoop to classpath
Adding Hive conf dir /opt/meituan/spark-2.2/conf to classpath
Creating SparkSession


log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@32b5a4e0
spark: SparkSession = org.apache.spark.sql.SparkSession@7f587cc9
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res0_16: String = "application_1671244879475_22023371"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

// 通用udf注册

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import o

In [2]:
var user="wanghao251"
var train_table_name ="mart_waimaiad.AFO_RL_1673942705568"
var predict_table_name="mart_waimaiad.AFO_RL_1673842737001"

var reward="reward"//RL reward 字段名字
var poi_id="poi_id"// pv 后续会去掉，修改为key
var pvid="pvid"// pv 后续会去掉，修改为key
var action="action"//RL action 字段名字
var is_terminal="is_terminal" //RL terminal 字段
var path_prefix=s"viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/${user}" //模型和tf recoder位置


var start_date="20230115" 
var end_date="20230115" 

var trainDt=s"""pt=20230116"""
var predictDt=s"""pt=20230115"""

var model_name="R-BCQ" //是否使用REM

var task_type="low"
var use_bcorle="True"
var use_adaptive="False"
var use_batch_loss="False"
var use_mask="True"
var lambda_alpha="1e-1_1e-1_1e-1_1e-1"
var lambda_update_interval="100"
var lambda_budgets_target= "1_1_1_1"
var constraint_target="1_1_1_1"
var constraint_loss_weight="0.01_0.01_0.01_0.01"

var high_model_dir="viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230112/REM_high_data_0114_constraint/high"
var high_mean_var_dir="/opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673613376579/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20230112"

var high_state_dynamic_fea="ctr,histctr,aimcpc,impr,cpc,revenue,click,order_num,mt_butie,impr_before1week,click_before1week,cpc_before1week,order_num_before1week,revenue_before1week,mt_butie_before1week,charge_before1week,impr_before2week,click_before2week,cpc_before2week,revenue_before2week,charge_before2week,order_num_before2week,mt_butie_before2week,impr_before3week,cpc_before3week,click_before3week,order_num_before3week,mt_butie_before3week,revenue_before3week,charge_before3week,impr_before4week,charge_before4week,revenue_before4week,mt_butie_before4week,cpc_before4week,click_before4week,order_num_before4week".toLowerCase()
var low_state_dynamic_fea="ctr,histctr,aimcpc,impr,cpc,revenue,click,order_num,mt_butie,impr_before1week,click_before1week,cpc_before1week,order_num_before1week,revenue_before1week,mt_butie_before1week,charge_before1week,impr_before2week,click_before2week,cpc_before2week,revenue_before2week,charge_before2week,order_num_before2week,mt_butie_before2week,impr_before3week,cpc_before3week,click_before3week,order_num_before3week,mt_butie_before3week,revenue_before3week,charge_before3week,impr_before4week,charge_before4week,revenue_before4week,mt_butie_before4week,cpc_before4week,click_before4week,order_num_before4week,user_tag_pref_click_3day,user_ord_num_weekdays,feacomp_cvr,lowerk,real_expose_all,real_cpc_all,real_csm_all,real_revenue_all,real_order_all,real_click_all,real_cst_bdgt,real_aBln,real_price,real_budget,real_csm_bln,real_bln,winningscore_comp12,feacomp_ctr,ratioctr,lowerbid,ratio_aimcpc,winningscore_qs,ratio_click,ratioctr_exp,upperk,charge,ratio_cpc,real_csm,real_expose,real_cpc,real_click,real_cash,real_order,real_revenue".toLowerCase()
var high_state_cate_fea="poi_id"
var low_state_cate_fea="poi_id,cityid,hour"

var parameter_servers=1 //ps的数量
var workers=2 //work的数量
//var parameter_servers=1 //ps的数量
//var workers=2 //work的数量
var batch_size="1024" // batch size
var epoch="1"

var account="wanghao251" //AFO框架启动消息通知对象
var queue="root.zw05_training_cluster.hadoop-waimaiadrd.cpu_job"
// 模型配置区

// var batch_size="128"

var learning_rate="0.00001"
var deep_layers="128,64,64" //网络设置
var gamma="0.995"
var num_action="20" //action的个数
var update_interval="2000"
var embed_dim="8" //离散特征的embedding维度
var bit="22"//离散特征的空间
var ext_is_predict_serving="0"
var use_bn="False"
var i_loss_weight="1" //使用BCQ时iloss的权重
var i_regularization_weight="0.01" 
var q_loss_weight="1" //使用BCQ时qloss的权重
var num_heads="20" //REM head 个数
var threshold="0.3" // BCQ阈值
var use_cate_feature="True"
var use_dense_hashtable="True" //是否使用dense_hashtable
// var cur_state_cate_fea="poi_id,rn" //RL 状态特征
// var next_state_cate_fea="next_poi_id,next_rn" //RL 下一个状态特征
var model_file="udm/tangbo17/rbcq.py" //用户自定义模型代码文件
var readFromHive="True"
var file_path="allinone" // 模型的文件目录
var mod="local" //mod=local 代表启动workbench本地调试，其他代表提交到服务器
var code_branch = "master"

user: String = "wanghao251"
train_table_name: String = "mart_waimaiad.AFO_RL_1673942705568"
predict_table_name: String = "mart_waimaiad.AFO_RL_1673842737001"
reward: String = "reward"
poi_id: String = "poi_id"
pvid: String = "pvid"
action: String = "action"
is_terminal: String = "is_terminal"
path_prefix: String = "viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251"
start_date: String = "20230115"
end_date: String = "20230115"
trainDt: String = "pt=20230116"
predictDt: String = "pt=20230115"
model_name: String = "R-BCQ"
task_type: String = "low"
use_bcorle: String = "True"
use_adaptive: String = "False"
use_batch_loss: String = "False"
use_mask: String = "True"
lambda_alpha: String = "1e-1_1e-1_1e-1_1e-1"
lambda_update_interval: String = "100"
lambda_budgets_target: String = "1_1_1_1"
constraint_target: String = "1_1_1_1"
constraint_loss_weight: String = "0.01_0.01_0.01_0.01"
high_model_dir: String = "viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/202301

In [4]:

var task="train" //任务类型 train,predict
var with_chief="true"
var remark="low_0115_mask_bcole"

mod="local1" 
var train_cmd=s"""
    if [[ ! -d "/opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_${user}/${train_table_name}
        cd /opt/meituan/dolphinfs_${user}/${train_table_name}
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout ${code_branch}
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter
    git pull
    ls
    cd OS/wanghao/${file_path}
    echo "开始train model"
    cp ${high_mean_var_dir} ./data/
    sh run.sh ${train_table_name} ${predict_table_name} ${model_name} ${batch_size} ${epoch} ${learning_rate} ${deep_layers} ${gamma} ${num_action} ${update_interval} ${embed_dim} ${bit} ${ext_is_predict_serving} ${use_bn} ${i_loss_weight} ${i_regularization_weight} ${q_loss_weight} ${num_heads} ${threshold} ${path_prefix} ${start_date} ${end_date} ${high_state_dynamic_fea} ${low_state_dynamic_fea} ${task} ${parameter_servers} ${workers} ${account} ${queue} ${with_chief} ${use_dense_hashtable} ${high_state_cate_fea} ${low_state_cate_fea} ${use_cate_feature} ${readFromHive} ${file_path} ${mod} ${train_table_name}/${trainDt} ${use_adaptive} ${use_batch_loss} ${lambda_alpha} ${lambda_update_interval} ${lambda_budgets_target} ${constraint_target} ${constraint_loss_weight} ${use_mask} ${use_bcorle} ${task_type} ${high_model_dir} ${remark} ${high_mean_var_dir}
    """
executeShell(train_cmd)


2023-01-18 13:25:46
----------------The content of shell is: -------------------

    if [[ ! -d "/opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568
        cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout master
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter
    git pull
    ls
    cd OS/wanghao/allinone
    echo "开始train model"
    cp /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673613376579/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20230112 ./data/
    sh run.sh mart_waimaiad.AFO_RL_1673942705568 mart_waimaiad.AFO_RL_1673842737001 R-BCQ 1024 1 0.00001 128,64,64 0.995 20 2000 8 22 0 False 1 0.01 1 20 0.3 viewfs://hadoop-meituan/us

美团代码仓库(DevTools Code)                                 代码仓库组
[连接信息]:         zf-proxy01 => 5475d139-758c-4080-8463-2665bc895f9b
[路由信息]:                                 slave-mcode_shard_03-zf-06
🏮🏮━︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵︵━🏮🏮
 / **･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･゜ﾟ･･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･゜\
  |        辞旧迎新，DevTools Code团队祝童鞋们2023兔年快乐!         |
 \ *･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･゜ﾟ･*･゜ﾟ･*:.｡..｡.:*･･*:.｡. .｡.:*･｡./
  -------------------------------------------------------------------
                   ❄️        🎉阖家欢乐🎉        ❄️              
          卯         ❄️                        ❄️            兔 
          門        ❄️                          ❄️           歲 
          生     ❄️                                ❄️        報 
          喜       ❄️   Happy Spring Festival!   ❄️          新 
          氣         ❄️                        ❄️            春
               >>>>>> Better Decade Better Code <<<<<<
From ssh://git.sankuai.com/~tangbo17/mtjupter
   3da6518..0f7e300  master     ->

Updating 3da6518..0f7e300
Fast-forward
 OS/wanghao/allinone/rl_code/main.py | 2 +-
 OS/wanghao/allinone/run.sh          | 4 ++--
 2 files changed, 3 insertions(+), 3 deletions(-)
common
OS
personal
PT
readme.md
RecCpc
test.py
tool
开始train model


+ set -u
+ set -o pipefail
+ table_name=mart_waimaiad.AFO_RL_1673942705568
+ predict_table_name=mart_waimaiad.AFO_RL_1673842737001
+ algorithm_name=R-BCQ
+ batch_size=1024
+ epoch=1
+ learning_rate=0.00001


use_bcq:True
use_rem:True


...

+ HADOOP_HOME=/opt/meituan/hadoop
+ config_xml=dist.xml
+ is_dist=1
+ export HADOOP_HOME=/opt/meituan/hadoop-gpu
+ HADOOP_HOME=/opt/meituan/hadoop-gpu
+ export AFO_TF_HOME=/opt/meituan/tensorflow-release
+ AFO_TF_HOME=/opt/meituan/tensorflow-release
+ tensorflow_submit=/opt/meituan/tensorflow-release/bin/tensorflow-submit.sh
+ train_data=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/train_tfrecoder/20230115
+ predict_data=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
+ predict_result_path=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_result/20230115/R-BCQ_low_0115_mask_bcole
+ echo 'train_data: viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/train_tfrecoder/20230115'
+ echo 'predict_data: viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115'
+ echo 'predict_result_path: viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_result/20230115/

train_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/train_tfrecoder/20230115
predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
ext_is_predict_serving:0


e'
+ echo 'origin task:train'
+ case ${task} in
+ train_data=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/train_tfrecoder/20230115
+ ext_is_predict_serving=0
+ echo 'new task:train'
+ echo train_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/train_tfrecoder/20230115
+ echo predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
+ echo ext_is_predict_serving:0
+ case ${mod} in
+ hope run --xml conf/dist.xml --usergroup hadoop-hmart-waimaiad -Dworker.memory=102400 -Dafo.engine.wait_for_job_finished=true -Ddistribute.mode=ps '-Dworker.script=python rl_code/main.py' '-Dparameter.server.script=python rl_code/main.py' '-Dchief.script=python rl_code/main.py' -Dboard.log_dir=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230115/R-BCQ_low_0115_mask_bcole/low -Dparameter.servers=1 -Dargs.ext_is_predict_serving=0 -Dworkers=2 -Dwith.chief=true -Dafo.xm.notice.receivers.account=wanghao251 -Dafo

2023-01-18 13:25:49 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter/OS/wanghao/allinone/conf/dist.xml
2023-01-18 13:25:49 [INFO] Job info initialized at /workdir/sankuai/.hope/template_jobinfo/a98ddcadc3f03f852b43c4edf6969d55
2023-01-18 13:25:50 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter/OS/wanghao/allinone
2023-01-18 13:25:50 [INFO] Hope config check success
2023-01-18 13:25:50 [INFO] File name: /workdir/sankuai/.hope/job-91fdd402_96f0_11ed_b903_161ba065b71f/dist.tar.gz
2023-01-18 13:25:50 [INFO] File size: 108294B
2023-01-18 13:25:50 [INFO] Uploading...
2023-01-18 13:25:50 [INFO] Upload success
**********************************
 [INFO] code version save success 
**********************************
*************************************
 job submit success, run_id: 1156849 
*************************************
download file: http://s3plus.vip.sankuai.com/v

: 

In [ ]:
var task="predict" //任务类型为predict 时 with_chief="false" epoch="1"
var with_chief="false"
var workers=1
var parameter_servers=0
var epoch="1"
var batch_size=64
var mod="local1" 

var remark="low_0115_normal_mask2"

var train_cmd=s"""
    if [[ ! -d "/opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_${user}/${train_table_name}
        cd /opt/meituan/dolphinfs_${user}/${train_table_name}
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout ${code_branch}
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter
    git pull
    ls
    cd OS/wanghao/${file_path}
    echo "开始train model"
    cp ${high_mean_var_dir} ./data/
    sh run.sh ${train_table_name} ${predict_table_name} ${model_name} ${batch_size} ${epoch} ${learning_rate} ${deep_layers} ${gamma} ${num_action} ${update_interval} ${embed_dim} ${bit} ${ext_is_predict_serving} ${use_bn} ${i_loss_weight} ${i_regularization_weight} ${q_loss_weight} ${num_heads} ${threshold} ${path_prefix} ${start_date} ${end_date} ${high_state_dynamic_fea} ${low_state_dynamic_fea} ${task} ${parameter_servers} ${workers} ${account} ${queue} ${with_chief} ${use_dense_hashtable} ${high_state_cate_fea} ${low_state_cate_fea} ${use_cate_feature} ${readFromHive} ${file_path} ${mod} ${predict_table_name}/${predictDt} ${use_adaptive} ${use_batch_loss} ${lambda_alpha} ${lambda_update_interval} ${lambda_budgets_target} ${constraint_target} ${constraint_loss_weight} ${use_mask} ${use_bcorle} ${task_type} ${high_model_dir} ${remark} ${high_mean_var_dir}
    """
executeShell(train_cmd)


2023-01-17 22:45:55
----------------The content of shell is: -------------------

    if [[ ! -d "/opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        mkdir /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568
        cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568
        git clone ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
        git checkout master
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter
    git pull
    ls
    cd OS/wanghao/allinone
    echo "开始train model"
    cp /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673613376579/mtjupter/OS/wanghao/allinone/data/high_mean_var_of_state.20230112 ./data/
    sh run.sh mart_waimaiad.AFO_RL_1673942705568 mart_waimaiad.AFO_RL_1673842737001 R-BCQ 64 1 0.00001 128,64,64 0.995 20 2000 8 22 0 False 1 0.01 1 20 0.3 viewfs://hadoop-meituan/user

美团代码仓库(DevTools Code)                                 代码仓库组
[连接信息]:         gh-proxy02 => 4673dfe6-9bbd-44a9-b16b-a676037016da
[路由信息]:                                 slave-mcode_shard_03-gh-01
                                   


Already up-to-date.
common
OS
personal
PT
readme.md
RecCpc
test.py
tool
开始train model


+ set -u


use_bcq:True
use_rem:True


...

new task:predict


+ echo 'new task:predict'
+ echo train_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
+ echo predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
+ echo ext_is_predict_serving:0


train_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
predict_data:viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_tfrecoder/20230115
ext_is_predict_serving:0


+ case ${mod} in
+ hope run --xml conf/dist.xml --usergroup hadoop-hmart-waimaiad -Dworker.memory=102400 -Dafo.engine.wait_for_job_finished=true -Ddistribute.mode=ps '-Dworker.script=python rl_code/main.py' '-Dparameter.server.script=python rl_code/main.py' '-Dchief.script=python rl_code/main.py' -Dboard.log_dir=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230115/R-BCQ_low_0115_normal_mask2/low -Dparameter.servers=0 -Dargs.ext_is_predict_serving=0 -Dworkers=1 -Dwith.chief=false -Dafo.xm.notice.receivers.account=wanghao251 -Dafo.app.name=R-BCQ_low_0115_normal_mask2 -Dqueue=root.zw05_training_cluster.hadoop-waimaiadrd.cpu_job -Dargs.log_dir=viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230115/R-BCQ_low_0115_normal_mask2/low -Dargs.high_state_dynamic_fea_mean_var_filename=data/low_mean_var_of_state.20230115 -Dargs.low_state_dynamic_fea_mean_var_filename=data/low_mean_var_of_state.20230115 -Dargs.task=predict -Dargs.is_dist=1 -Dargs.batch_si

2023-01-17 22:46:04 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter/OS/wanghao/allinone/conf/dist.xml
2023-01-17 22:46:04 [INFO] Job info initialized at /workdir/sankuai/.hope/template_jobinfo/a98ddcadc3f03f852b43c4edf6969d55
2023-01-17 22:46:04 [INFO] Staged file or directory: /opt/meituan/dolphinfs_wanghao251/mart_waimaiad.AFO_RL_1673942705568/mtjupter/OS/wanghao/allinone
2023-01-17 22:46:04 [INFO] Hope config check success
2023-01-17 22:46:04 [INFO] File name: /workdir/sankuai/.hope/job-ab381a20_9675_11ed_a717_161ba065b71f/dist.tar.gz
2023-01-17 22:46:04 [INFO] File size: 108192B
2023-01-17 22:46:04 [INFO] Uploading...
2023-01-17 22:46:04 [INFO] Upload success
**********************************
 [INFO] code version save success 
**********************************
*************************************
 job submit success, run_id: 1151701 
*************************************
download file: http://s3plus.vip.sankuai.com/v

In [5]:

executeShell(s"""
hope exec --jobname hadoop-hmart-waimaiad.spark.os_bid_framework --args="20230117 gen /ghnn01/user/hadoop-waimai/maoshangqin/evaFl_os/  viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230102/R-BCQ_low_data_0102_real_666_clip/low"
""")

2023-01-18 13:26:09
----------------The content of shell is: -------------------

hope exec --jobname hadoop-hmart-waimaiad.spark.os_bid_framework --args="20230117 gen /ghnn01/user/hadoop-waimai/maoshangqin/evaFl_os/  viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/model/20230102/R-BCQ_low_data_0102_real_666_clip/low"

----------------Start execution....-------------------
* * * * * * * * * * * * * * * * * * * * 
Job Execute ID:	 28739198
* * * * * * * * * * * * * * * * * * * * 
hadoop-hmart-waimaiad.spark.os_bid_framework/12688598
2023-01-18 13:26:12 [Console] INFO Fetching file from http://etl.xt.vip.sankuai.com/exapi/sw/task/code/hadoop-hmart-waimaiad.spark.os_bid_framework/12688598
2023-01-18 13:26:12 [Console] INFO Fetching finish
hadoop-default
Task File: /opt/tmp/etl/remote_file/session_936D7B2405F1334A/hadoop-hmart-waimaiad.spark.os_bid_framework
Flow ID: 936D7B2405F1334A
Current Time: 2023-01-18 13:26:10
Delta Now: 2023-01-18 13:26:10
{'step0': {'class': 'com.san

In [ ]:
var predict_path = "viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/predict_result/20230102/R-BCQ_low_data_0102_0105"
executeShell(s"""
hope exec --jobname hadoop-hmart-waimaiad.spark.os_bid_framework --args="20230102 load_eval_rlk /ghnn01/user/hadoop-waimai/maoshangqin/evaFl_os/ ${predict_path}"
""")

In [ ]:
var tmpTrainTable = createTableFromSQL(s"""
select dt,poi_id,pvid,rawbid,precisebid,is_expose,is_click,final_charge,is_order,revenue,unused,click,user_tag_pref_click_3day,hour,user_ord_num_weekdays,charge_before2week,order_num_before2week,impr_before1week,feacomp_cvr,click_before1week,lowerk,revenue_before3week,cpc_before1week,charge_before3week,winningscore_comp12,impr,feacomp_ctr,real_expose_all,real_cpc_all,real_csm_all,real_cst_bdgt,real_aBln,real_revenue_all,real_price,real_budget,real_csm_bln,real_click_all,real_order_all,real_bln,revenue_before2week,ratioctr,lowerbid,ratio_aimcpc,cpc_before4week,order_num_before3week,histctr,mt_butie_before2week,mt_butie_before3week,order_num_before4week,cpc,order_num_before1week,revenue_before1week,mt_butie,cpc_before2week,cpc_before3week,aimcpc,mt_butie_before1week,click_before3week,charge_before1week,click_before4week,click_before2week,winningscore_qs,cityid,ratio_click,prod,ratioctr_exp,upperk,charge,revenue_before4week,ctr,order_num,impr_before4week,impr_before2week,mt_butie_before4week,ratio_cpc,impr_before3week,charge_before4week,real_cpc,real_csm,real_click,real_cash,real_order,real_revenue,real_expose,next_real_click,next_real_cpc,next_real_csm,next_real_cash,next_real_order,next_real_revenue,next_real_expose,next_rawbid,next_real_expose_all,next_precisebid,next_is_expose,next_is_click,next_final_charge,next_is_order,next_revenue,next_unused,next_click,next_user_tag_pref_click_3day,next_hour,next_user_ord_num_weekdays,next_charge_before2week,next_order_num_before2week,next_impr_before1week,next_feacomp_cvr,next_click_before1week,next_lowerk,next_revenue_before3week,next_cpc_before1week,next_charge_before3week,next_winningscore_comp12,next_impr,next_feacomp_ctr,next_revenue_before2week,next_ratioctr,next_lowerbid,next_ratio_aimcpc,next_cpc_before4week,next_order_num_before3week,next_histctr,next_mt_butie_before2week,next_mt_butie_before3week,next_real_cpc_all,next_real_csm_all,next_real_revenue_all,next_real_click_all,next_real_order_all,next_real_cst_bdgt,next_real_aBln,next_real_price,next_real_budget,next_real_csm_bln,next_real_bln,next_order_num_before4week,next_cpc,next_order_num_before1week,next_revenue_before1week,next_mt_butie,next_cpc_before2week,next_cpc_before3week,next_aimcpc,next_mt_butie_before1week,next_click_before3week,next_charge_before1week,next_click_before4week,next_click_before2week,next_winningscore_qs,next_cityid,next_ratio_click,next_ratioctr_exp,next_upperk,next_charge,next_revenue_before4week,next_ctr,next_order_num,next_impr_before4week,next_impr_before2week,next_mt_butie_before4week,next_ratio_cpc,next_impr_before3week,next_charge_before4week,rlkAct,k_dj,k_bj,k_ss,k_push,reward,is_terminal,action_pre_dj,action_pre_bj,action_pre_ss,action_pre_push,action from mart_waimaiad.AFO_RL_1673840629901 limit 100000
""")

In [ ]:
var finalTrainTable= createPTTableFromTableWithSpace(tmpTrainTable,"mart_waimaiad") //正式使用请按照自己的逻辑生成，表的主键是pvid，poi_id

In [10]:
showTopNFromSQL("""
select real_click,real_csm,real_cash,aimcpc,real_budget,real_csm_all from mart_waimaiad.AFO_RL_1673943935631 where real_click[0]>10
""",2)

2023-01-18 11:04:03
2023-01-18 11:04:03
执行 
 
select real_click,real_csm,real_cash,aimcpc,real_budget,real_csm_all from mart_waimaiad.AFO_RL_1673943935631 where real_click[0]>10

2023-01-18 11:04:03


show at cmd0.sc:161

1 / 1

+---------------------+--------------------------+--------------------------------+----------------------+------------------------+-----------------------+
|real_click           |real_csm                  |real_cash                       |aimcpc                |real_budget             |real_csm_all           |
+---------------------+--------------------------+--------------------------------+----------------------+------------------------+-----------------------+
|[48.0, 0.0, 9.0, 0.0]|[5578.0, 0.0, 1064.0, 0.0]|[4462.4004, 0.0, 851.19995, 0.0]|[0.0, 0.0, 100.0, 0.0]|[0.0, 0.0, 35000.0, 0.0]|[0.0, 0.0, 0.0, 0.0]   |
|[26.0, 0.0, 4.0, 0.0]|[1246.22, 0.0, 190.0, 0.0]|[1246.22, 0.0, 190.0, 0.0]      |[0.0, 0.0, 0.0, 50.0] |[0.0, 0.0, 0.0, 20000.0]|[0.0, 0.0, 0.0, 1333.0]|
+---------------------+--------------------------+--------------------------------+----------------------+------------------------+-----------------------+
only showing top 2 rows

2023-01-18 11:04:13
